In [ ]:
# 필요 Library install
!pip install transformers==4.39.2 peft==0.10.0 trl==0.8.6 bitsandbytes==0.43.0 accelerate==0.29.0

In [ ]:
# HF token 설정
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 모델 경량화: Quantization 설정
from transformers import BitsAndBytesConfig
import torch

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [ ]:
# 모델 경량화: Lora 설정
from peft import LoraConfig
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj","k_proj","o_proj","gate_proj","up_proj","down_proj"]
)

In [ ]:
# 기본 Llama 3 모델 로드
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config = quantization_config,
    device_map = {"": 0}
)

In [ ]:
# Tokenizer 설정
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_250|>"})
model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
# Prompt/Response Format 관련 설정
EOS_TOKEN = tokenizer.eos_token

def convert_to_alpaca_format(instruction, response):
    alpaca_format_str = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\
    \n\n### Instruction:\n{instruction}\n\n### Response:\n{response}\
    """

    return alpaca_format_str


def prompt_formatting_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        alpaca_formatted_str = convert_to_alpaca_format(instruction, output) + EOS_TOKEN
        texts.append(alpaca_formatted_str)
    return { "text" : texts, }

In [ ]:
# Dataset Load
from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")

dataset = dataset.shuffle(seed=42)
no_input_dataset = dataset.filter(lambda example: example['input'] == '')
mapped_dataset = no_input_dataset.map(prompt_formatting_func, batched=True)
split_dataset = mapped_dataset.train_test_split(test_size=0.01, seed=42)

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
# Data Collator 설정
from trl import DataCollatorForCompletionOnlyLM
data_collator_param = {}
response_template = "### Response:\n"
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer, mlm=False)
data_collator_param["data_collator"] = collator

In [ ]:
# local output dir 설정
local_output_dir = "/content/fine_tune_output"

In [ ]:
!mkdir {local_output_dir}

In [ ]:
# tensorboard 설정
%load_ext tensorboard
%tensorboard --logdir '{local_output_dir}/runs'

In [ ]:
# Training setup
from trl import SFTTrainer
from transformers import TrainingArguments

training_arguments = TrainingArguments(
  output_dir=local_output_dir,
  report_to = "tensorboard",
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 2,
  gradient_accumulation_steps = 8,
  warmup_steps = 50,
  max_steps = 100,
  eval_steps=10,
  save_steps=50,
  evaluation_strategy="steps",
  save_strategy="steps",
  learning_rate = 1e-4,
  logging_steps = 1,
  optim = "adamw_8bit",
  weight_decay = 0.01,
  lr_scheduler_type = "constant_with_warmup",
  seed = 42,
  gradient_checkpointing = True,
  gradient_checkpointing_kwargs={'use_reentrant':True}
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    peft_config=peft_config,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = training_arguments,
    **data_collator_param
)

In [ ]:
train_stats = trainer.train()

In [ ]:
# Google drive로 복사
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r {local_output_dir} /content/drive/MyDrive